# 3.2 Fun with Rekognition DetectFaces and SearchFacesByImage
----
This is fun but optional module. Here we are going to take a look at extracting and indexing multiple faces, from an image. Once we have the collection of images we'll compare all faces in an image with Amazon Rekognition against your collection.

## Steps 

1. Load packages 
2. View existing collections 
3. Create a new collection 
4. Download some images and unzip them into a directory 
5. Index the faces found into the collection 
6. Find and present faces found in an image vs. the faces stored in your collection 
7. Clean up!! 


The following notebook will guide you through on how to compare all faces detected in an image against your Amazon Rekognition Face Collection. 

## Step 1. Load Libraries 

In [ ]:
import boto3, json, os, io
from PIL import Image, ImageDraw, ExifTags, ImageColor
from IPython.display import Image as IImage
client=boto3.client('rekognition')

## Step 2. View your existing collections

In [ ]:
def list_collections():

    max_results=10
    
    #Display all the collections
    print('Displaying collections...')
    response=client.list_collections(MaxResults=max_results)
    collection_count=0
    done=False
    
    while not done:
        collections=response['CollectionIds']

        for collection in collections:
            print (collection)
            collection_count+=1
        if 'NextToken' in response:
            nextToken=response['NextToken']
            response=client.list_collections(NextToken=nextToken,MaxResults=max_results)
            
        else:
            done=True

    return collection_count   

collection_count=list_collections()
print("collections: " + str(collection_count))


## Step 3. Create a new collection
-----

Remember you must use a unique name if you are creating a new collection

In [ ]:
collection_id='Avengers_Collection' # Remember you must use a unique name if you are creating a new collection

In [ ]:
def create_collection(collection_id):
    #Create a collection
    print('Creating collection:' + collection_id)
    try:
        response=client.create_collection(CollectionId=collection_id)
    except:
        client.delete_collection(CollectionId=collection_id)
        response=client.create_collection(CollectionId=collection_id)
    print('Collection ARN: ' + response['CollectionArn'])
    print('Status code: ' + str(response['StatusCode']))
    print('Done...')
    
create_collection(collection_id)

### Step 3a. Confirm your collection creation. 
-----


In [ ]:
collection_count=list_collections()
print("collections: " + str(collection_count))

## Step 4. Download collection images
---
here we are going to download a zip file of images, unzip it and then index it. 

In [ ]:
!rm -R avengers_face_search
!curl https://dz4iesrkqs7j9.cloudfront.net/random/avengers_face_search.zip -O
!unzip avengers_face_search.zip
!rm -R __MACOSX
!rm avengers_face_search.zip

## Step 5. Index faces (add faces to a collection) 
-----
Here we are going to iterate over the files in the population folder and index their faces. 

In [ ]:
directory = 'avengers_face_search/populate'
 
# iterate over files in
# that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    with open(f, "rb") as file:
        print(f)
        file = open(f, "rb") # opening for [r]eading as [b]inary
        data = file.read() 
        response=client.index_faces(CollectionId=collection_id,
                            Image={'Bytes':data},
                            ExternalImageId=f.split("/")[2],
                            MaxFaces=1,
                            QualityFilter="AUTO",
                            DetectionAttributes=['ALL'])
        print ('Results for ' + f.split("/")[2])
        print('Faces indexed:')
        for faceRecord in response['FaceRecords']:
            print('  Face ID : ' + faceRecord['Face']['FaceId'])
            print('  Location: {}'.format(faceRecord['Face']['BoundingBox']))

        print('Faces not indexed:')
        for unindexedFace in response['UnindexedFaces']:
            print(' Location: {}'.format(unindexedFace['FaceDetail']['BoundingBox']))
            print(' Reasons :')
            for reason in unindexedFace['Reasons']:
                print('   ' + reason)
        file.close()


### 5a. List faces in the collection

In [ ]:
def list_faces_in_collection(collection_id):

    maxResults=20
    faces_count=0
    tokens=True

    response=client.list_faces(CollectionId=collection_id,
                               MaxResults=maxResults)

    print('Faces in collection ' + collection_id)
 
    while tokens:

        faces=response['Faces']
        for face in faces:
            print (face["FaceId"],face["ExternalImageId"].split('.')[0])
            faces_count+=1
        if 'NextToken' in response:
            nextToken=response['NextToken']
            response=client.list_faces(CollectionId=collection_id,
                                       NextToken=nextToken,MaxResults=maxResults)
        else:
            tokens=False
    return faces_count   

faces_count=list_faces_in_collection(collection_id)
print("Faces in collection: " + str(faces_count))

## Step 6. Find faces in photo

----
here we create a few functions that will be useful for transforming and detecting faces 

In [ ]:
def transform_bounding(img, box):
    imgWidth, imgHeight = img.size
    l = (imgWidth * box['Left'])-5
    t = (imgHeight * box['Top'])-5
    w = (imgWidth * box['Width'])+10
    h = (imgHeight * box['Height'])+10
    return l,t,w,h

In [ ]:
def detect_faces(file):
    faces = []
    f = open(file, "rb") # opening for [r]eading as [b]inary
    data = f.read() 
    response = client.detect_faces(Image={'Bytes':data})
    for face in response["FaceDetails"]:
        faces.append(face['BoundingBox'])
    print("Faces detected: " + str(len(response['FaceDetails'])))   
    return faces

In [ ]:
directory = 'avengers_face_search/test'

from PIL import Image               # to load images
from IPython.display import display # to display images

for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    with open(f, "rb") as file:
        img = Image.open(f)
        display(img)
        faces = detect_faces(f)        
        for face in faces:
            l,t,w,h = transform_bounding(img,face)
            cropped = img.crop((l,t,l+w,t+h))    

            stream = io.BytesIO()
            cropped.save(stream, format='JPEG')
            bin_img = stream.getvalue()

            response0 = client.detect_faces(
                Image={'Bytes': bin_img},
            )

            if len(response0['FaceDetails']) > 0:
                print("face found")
                display(cropped)
                response1=client.search_faces_by_image(CollectionId="Avengers_Collection",
                                                  Image={'Bytes': bin_img},
                                                  FaceMatchThreshold=50)
                faceMatches=response1['FaceMatches']
                if(len(faceMatches) > 0):
                    for match in faceMatches:
                        print ('Character  : ' + match['Face']['ExternalImageId'].split('.')[0])
                        print ('Similarity : ' + "{:.2f}".format(match['Similarity']) + "%")
                else:
                    print("but no match found")
            else:
                print("face not found in the following crop")
                cropped.show()

            print("------------------------------")  

## Clean up the resources

Delete your face collection 

In [ ]:
def delete_collection(collection_id):


    print('Attempting to delete collection ' + collection_id)
    client=boto3.client('rekognition')
    status_code=0
    try:
        response=client.delete_collection(CollectionId=collection_id)
        status_code=response['StatusCode']
        
    except ClientError as e:
        if e.response['Error']['Code'] == 'ResourceNotFoundException':
            print ('The collection ' + collection_id + ' was not found ')
        else:
            print ('Error other than Not Found occurred: ' + e.response['Error']['Message'])
        status_code=e.response['ResponseMetadata']['HTTPStatusCode']
    return status_code


def main():
    collection_id='Avengers_Collection'
    status_code=delete_collection(collection_id)
    print('Status code: ' + str(status_code))

if __name__ == "__main__":
    main()  